## A Jupyter Notebook widget to plot shiptrack alongside data

data is ONLY for demonstration purposes. If you would like to use the data for any kind of analysis, please contact flovan@bas.ac.uk first

In [10]:
from IPython.display import Image, display, Math, Latex, HTML, Javascript
display(Javascript('IPython.notebook.execute_cells_below()'))

<IPython.core.display.Javascript object>

In [11]:
# TOGGLE ON/OFF RAW CODE
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')



In [19]:
from mpl_toolkits.basemap import Basemap

import numpy as np
import matplotlib.pyplot as plt
import datetime as dt

from netCDF4 import Dataset

%matplotlib inline
import os
import sys
import gzip
import pickle

from mypysmps.util.timetransform import TimeTransform
tt = TimeTransform()

from mypysmps.io.read import read
from mypysmps.util.ps_utils import stack_ps
from mypysmps.graph.psdisplay import PSDisplay

from ipywidgets import *
from IPython.display import display
from IPython.display import display, clear_output
import ipywidgets as widgets

import matplotlib

font = {'family' : 'normal',
        'weight' : 'normal',
        'size'   : 18}

matplotlib.rc('font', **font)

In [20]:
sys.path.append("./../") # set this path to where the mypysmps is located

In [23]:
%%capture
avOPC = pickle.load(gzip.open("../data/down.gz", 'rb'))

In [14]:
def getLatLons(OPC):
    tlons = []
    tlats = []
    for i in range(0, len(OPC.latitude['data'])):
        alat = OPC.latitude['data'][i]
        alon = OPC.longitude['data'][i]
        try:
            tlons.append( -(float(alon[0:2])+ (float(alon[2:].strip('W'))/60)) )
            tlats.append( -(float(alat[0:2])+ (float(alat[2:].strip('S'))/60)) )
        except ValueError:
            pass
        
    return tlons, tlats

In [15]:
tlons, tlats = getLatLons(avOPC)

In [16]:
def myplot2(opc, tlats, dateselect):
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize = (30,10), gridspec_kw={'width_ratios': [1.5, 3]})
    plt.sca(ax1)

    m = Basemap(projection='spstere',llcrnrlat=-10., llcrnrlon = -90., urcrnrlat=-10., urcrnrlon=0, boundinglat=-10,lon_0=180,resolution='l')

    m.drawcoastlines()

    _ = m.drawparallels(np.arange(-60.,1,15.))
    _ = m.drawmeridians(np.arange(0.,-91,-30.), labels = [True,True,True,True])

    _ = m.fillcontinents(color='black',lake_color='white')


    x, y = m(tlons[:dateselect],tlats[:dateselect])
    m.plot(x, y, 'g--', linewidth = 6)
    m.plot(x[0],y[0], 'bs', markersize=12)
    m.plot(x[-1],y[-1], 'ro', markersize=10)


    xtickloc = []

    for co in np.arange(-60.,1,15.):
        xtickloc.append( m(-90,co)[0] )

    xtickloc = xtickloc[::-1]
    plt.xticks(xtickloc,['0$^\circ$S',u'15\N{DEGREE SIGN}S',u'30\N{DEGREE SIGN}S',u'45\N{DEGREE SIGN}S',u'60\N{DEGREE SIGN}S'])


    llcrnrx,llcrnry = m(-90,-10)
    urcrnrx,urcrnry = m(0, -10)
    ax1.set_xlim([llcrnrx,urcrnrx])
    ax1.set_ylim([llcrnry,urcrnry])

    plt.sca(ax2)
    
    OPCsums = np.sum(opc.data['normalised_number_concentration']['data'], axis=0)
    ax2.plot(tlats, OPCsums)
    ylims = ax2.get_ylim()
    xvals = tlats[dateselect]

    ax2.plot([xvals,xvals], [ylims[0], 1.4*ylims[1]], 'r--')
    
    #ax2 = disp2.timeLine('normalised_number_concentration',indicator = 4788,return_axes=[fig,ax2],ylim = ylims,ylabel =  u'Diameter [\N{GREEK SMALL LETTER MU}m]')

    plt.subplots_adjust(wspace=0.05, hspace=0)


In [17]:
sample = widgets.IntSlider(min=1, max=len(avOPC.sample['data'])-1, continuous_update = False, description = 'sample #',layout = Layout(
            width = '100%'
        ))


In [18]:
psd = PSDisplay(avOPC)
date = widgets.interactive_output(psd.dateInfo,{'sample':sample})
outplot = widgets.interactive_output(myplot2, {'opc':fixed(avOPC),'tlats':fixed(tlats),'dateselect':sample})

uidate = HBox([date])
outpl1 = VBox([outplot,sample,date])
display(outpl1)